In [27]:
import os
import sys
import sqlite3
sys.path.insert(0, os.path.abspath('../'))  # add the current module so that we can import the utils file
from tools.utils import get_top_keywords_for_query, create_df_for_query

In [28]:
conn = sqlite3.connect('../aip.db')
start_year = 2009  # inclusive
end_year = 2018  # inclusive

In [29]:
# Get for each year, the top 100 keywords.
keywords_per_year = dict()
num_keywords = 10
# corpus_query = """
# SELECT * 
# FROM publications 
# WHERE year between ? and ?
# and (lower(title) like '%workflow%' or lower(abstract) like '%workflow%') 
# and (lower(title) like '%schedul%' or lower(abstract) like '%schedul%')
# """
corpus_query = """
SELECT * 
FROM publications 
WHERE year between ? and ?
"""
corpus_df = create_df_for_query(conn, corpus_query, [start_year, end_year])

In [30]:
for year in range(start_year, end_year + 1):
    query = "SELECT * FROM publications WHERE year = ?"  # the entire systems community
    # The workflow scheduling community
#     query = "SELECT * FROM publications WHERE year = ? and (lower(title) like '%workflow%' or lower(abstract) like '%workflow%') and (lower(title) like '%schedul%' or lower(abstract) like '%schedul%')"
    # the workflow formalism community
#     query = "SELECT * FROM publications WHERE year = ? AND (lower(title) LIKE '%workflow%' OR lower(abstract) LIKE '%workflow%') AND ((lower(title) LIKE '%formalism%' OR lower(abstract) LIKE '%formalism%') OR (lower(title) LIKE '%language%' OR lower(abstract) LIKE '%language%'))"
    # Workflow allocation
#     query = "SELECT * FROM publications WHERE (lower(title) LIKE '%workflow%' OR lower(abstract) LIKE '%workflow%') AND ((lower(title) LIKE '%allocat%' OR lower(abstract) LIKE '%allocat%') OR (lower(title) LIKE '%schedul%' OR lower(abstract) LIKE '%schedul%') OR (lower(title) LIKE '%plan%' OR lower(abstract) LIKE '%plan%')) AND year = ?"
    # Resource provisioning
#     query = "SELECT * FROM publications WHERE year = ? AND (lower(title) LIKE '%workflow%' OR lower(abstract) LIKE '%workflow%') AND (lower(title) LIKE '%provision%' OR lower(abstract) LIKE '%provision%' OR lower(title) LIKE '%autoscal%' OR lower(abstract) LIKE '%autoscal%')"
    # Service and appliations in clouds
#     query = " SELECT * FROM publications WHERE year = ? AND (lower(title) LIKE '%cloud%' OR lower(abstract) LIKE '%cloud%') AND (lower(title) LIKE '%service%' OR lower(abstract) LIKE '%service%')"
    
    keywords = get_top_keywords_for_query(conn, corpus_df, query, num_keywords, article_query_params=[year])
    keywords_per_year[year] = keywords

In [31]:
# This cell outputs keywords that are amoung the top-{num_keywords} in the past (end_year - last_years, end_year] years
# that are not found in the block [start_year, end_year - last_years]

keywords_in_the_last_years = set()
last_years = 5
for year in range(end_year, end_year - last_years, -1):
    for keyword in keywords_per_year[year]:
        keywords_in_the_last_years.add(keyword)

keywords_in_the_remaining_years = set()
for year in range(end_year - last_years, start_year - 1, -1):
    for keyword in keywords_per_year[year]:
        keywords_in_the_remaining_years.add(keyword)

emerging_keywords = keywords_in_the_last_years - keywords_in_the_remaining_years
print(sorted(emerging_keywords))

['proposed']


In [32]:
# This cell outputs keywords that have strictly risen in rank throughout the years in the top-{num_keywords}.

keyword_last_rank = dict()
seen_keywords = set()
emerging_keywords = set()

# Fill the dictionary with the initial year
for index in range(len(keywords_per_year[start_year])):
    keyword = keywords_per_year[start_year][index]
    keyword_last_rank[keyword] = index
    seen_keywords.add(keyword)
    emerging_keywords.add(keyword)
    
# for rank, word in keyword_last_rank.items():
#     print(rank, word)
# Now, we check each year's keywords. There can be four cases:
# 1. The keyword was also in last year's set. In this case the index (its rank) has to be higher or equal.
# 2. The keyword is not in keyword_last_rank _AND_ was never seen before, this means it can be an emerging field and should be added to the keyword_last_rank dictionary.
# 3. The keyword is not in keyword_last_rank _AND_ was seen before. This means it dropped throughout the years and should be removed.
# 4. A keyword was seen once and then never again, drop it if it was not encountered in the {end_year}.
for year in range(start_year + 1, end_year + 1):
    this_years_rank = dict()
    new_emerging_topics = set()  # Keep a clean set to automatically weed out keywords matching case 4
    for index in range(len(keywords_per_year[year])):
        keyword = keywords_per_year[year][index]
        
#         # For insight
#         if keyword == "part":
#             print(year, keyword, index)
        
        # Case 1
        if keyword in keyword_last_rank and keyword in emerging_keywords:
            if index <= keyword_last_rank[keyword]: # rank is lower, all is well
                this_years_rank[keyword] = index
                new_emerging_topics.add(keyword)
        elif keyword not in seen_keywords:  # Case 2
            this_years_rank[keyword] = index  # We can reuse keyword_last_rank since keywords are unique per year
            new_emerging_topics.add(keyword)
        else: # Case 3 - this else statement can be deleted, but for completion's sake we left it in
            if keyword in emerging_keywords:
                emerging_keywords.remove(keyword)
        
        seen_keywords.add(keyword)  # Always add the keyword to the seen keywords set.
        keyword_last_rank = this_years_rank  # Set the last year's rank dict to this year, to be ready for the next potential loop iteration.
        emerging_keywords = new_emerging_topics  # Since we swap at the end, keywords emerged in the last year are automatically kept.
        
print(emerging_keywords)

for word in emerging_keywords:
    for year in range(start_year, end_year + 1):
        if word in keywords_per_year[year]:
            print(word, year)

{'proposed'}
proposed 2018
